In [1]:
%matplotlib inline
import os
import ipyplot
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from models import isoefp_net
from training import train_model
from evaluation import make_dist
from data_processing import remove_excess_iso, shuffle_samples, \
    make_llp_boolean_labels, make_llp_iso_data_loaders, \
    iso_standard_scaler, select_cone_set, select_single_cone

In [25]:
# Assign parameters
seed = 1
num_epochs = 300
batch_size = 128
learning_rate = 1e-6
patience = 15
min_change = 5e-3
num_cones = 18
num_folds = 5
num_thresh = 100
data_dir = '../data'
train_dir = 'train'
test_dir = 'test'
image_folder = f'./images/iso_varied_denom_{num_thresh}_unique_thresh_{num_folds}_fold'
log_folder = f'./model_logs/iso_varied_denom_{num_thresh}_unique_thresh_{num_folds}_fold'

file_mu_label = 'mu1'
train_pos_prop_label = '95'
train_neg_prop_label = '20'
train_prop_label = f'{train_neg_prop_label}{train_pos_prop_label}'
test_pos_prop_label = '95'
test_neg_prop_label = '20'
test_prop_label = f'{test_neg_prop_label}{test_pos_prop_label}'

In [3]:
# Load data
x = np.load(f'{data_dir}/{train_dir}/{file_mu_label}_{train_prop_label}_isoset.npy')
f = np.load(f'{data_dir}/{train_dir}/{file_mu_label}_{train_prop_label}_labels.npy')
x_test = np.load(f'{data_dir}/{test_dir}/{file_mu_label}_{test_prop_label}_isoset.npy')
f_test = np.load(f'{data_dir}/{test_dir}/{file_mu_label}_{test_prop_label}_labels.npy')
m_test = np.load(f'{data_dir}/{test_dir}/{test_prop_label}_M_set.npy')

In [4]:
# Some empty events end up being loaded as they aren't empty for
# other muon in event and remain in dataset, get rid of these
nonzeros = np.sum(x,axis=1) > 0
nonzeros_test = np.sum(x_test,axis=1) > 0
x = x[nonzeros]
f = f[nonzeros]
x_test = x_test[nonzeros_test]
f_test = f_test[nonzeros_test]
m_test = m_test[nonzeros_test]

In [5]:
# Choose samples such that an integer number of batches can be made,
# ensures that each batch has uniform labels 
x, f = remove_excess_iso(x, f, batch_size, num_folds, num_cones, seed)
x, f, _ = shuffle_samples(x, f, batch_size, seed)

Labels: [0.2  0.95], Initial Respective Counts [88647 59035]
Labels: [0.2  0.95], Final Respective Counts [88320 58880]


In [6]:
# Create boolean labels from proportion labels (e.g. 0.2, 0.95 -> 0, 1)
y = make_llp_boolean_labels(f)
y_test = make_llp_boolean_labels(f_test)

In [7]:
# Make KFolds
kf = KFold(n_splits=num_folds, shuffle=False)

# Training

Train models for: 1 cone (largest) and 18 cones (all cones)

Recalculate FPR, TPR denominator at each threshold during AUC calculation, use ~100 unique outputs as thresholds

In [8]:
prefixes = []
cone_set_aucs = []
cone_set_sigs = []
for cone_count in [1,18]:
    fold_aucs = []
    for i, (train_index, valid_index) in enumerate(kf.split(x)):
        print(f'Fold {i+1} / {num_folds}, {cone_count} cones')
        
        train_dict = {'x': select_cone_set(x[train_index],cone_count), 
                      'f': f[train_index], 'y': y[train_index]}
        valid_dict = {'x': select_cone_set(x[valid_index],cone_count), 
                      'f': f[valid_index], 'y': y[valid_index]}
        test_dict = {'x': select_cone_set(x_test,cone_count),
                     'f': f_test, 'y': y_test, 'm': m_test}
        
        data_loaders = make_llp_iso_data_loaders(
            *iso_standard_scaler(train_dict, valid_dict, test_dict), 
            batch_size=batch_size, seed=seed)
        
        del train_dict, valid_dict, test_dict
        
        model = isoefp_net(cone_count,0)
        
        file_prefix = f'iso_varied_denom_{num_thresh}_unique_thresh_{cone_count}_cones_fold_{i}'
        image_prefix = f'{image_folder}/{file_prefix}'
        output_prefix = f'{log_folder}/{file_prefix}'
        
        model, log = train_model(model, data_loaders, llp=True, 
                num_epochs=num_epochs, learning_rate=learning_rate, seed=seed,
                reproducible=True, early_stopping=True, min_change=min_change,
                patience=patience, image_prefix=image_prefix, 
                output_prefix=output_prefix, auc_num_thresh=num_thresh,
                auc_const_denom=False)
        
        del data_loaders
        
        fold_aucs.append(log['test']['llp roc']['auc'])

    print(f'Average AUC: {np.mean(fold_aucs)} +- {np.std(fold_aucs)}')
    cone_set_aucs.append(np.mean(fold_aucs))
    cone_set_sigs.append(np.std(fold_aucs))

Fold 1 / 5, 1 cones
Using GPU
Epoch 1/300, Step 920/920, Loss: 0.0061
train_loss: -0.003, val_loss: -0.006, val_auc: 0.771

Epoch 2/300, Step 920/920, Loss: -0.006
train_loss: -0.010, val_loss: -0.015, val_auc: 0.771

Epoch 3/300, Step 920/920, Loss: -0.029
train_loss: -0.023, val_loss: -0.032, val_auc: 0.771

Epoch 4/300, Step 920/920, Loss: -0.057
train_loss: -0.042, val_loss: -0.053, val_auc: 0.771

Epoch 5/300, Step 920/920, Loss: -0.083
train_loss: -0.064, val_loss: -0.076, val_auc: 0.771

Epoch 6/300, Step 920/920, Loss: -0.104
train_loss: -0.086, val_loss: -0.096, val_auc: 0.771

Epoch 7/300, Step 920/920, Loss: -0.119
train_loss: -0.105, val_loss: -0.114, val_auc: 0.771

Epoch 8/300, Step 920/920, Loss: -0.128
train_loss: -0.121, val_loss: -0.129, val_auc: 0.771

Epoch 9/300, Step 920/920, Loss: -0.131
train_loss: -0.135, val_loss: -0.142, val_auc: 0.771

Epoch 10/300, Step 920/920, Loss: -0.130
train_loss: -0.148, val_loss: -0.154, val_auc: 0.771

Epoch 11/300, Step 920/920, L

/usr/lib/python3.9/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


/mnt/ntfs/home_backup/edwit4/Research/DoubleMuParked/cms_llp/llp_auc.py:360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  roc_dict[key] = np.array(roc_dict[key])


LLP AUC: 0.846237346585741
Fold 2 / 5, 1 cones
Using GPU
Epoch 1/300, Step 920/920, Loss: 0.0145
train_loss: -0.004, val_loss: -0.005, val_auc: 0.769

Epoch 2/300, Step 920/920, Loss: 0.0083
train_loss: -0.012, val_loss: -0.015, val_auc: 0.769

Epoch 3/300, Step 920/920, Loss: -0.006
train_loss: -0.024, val_loss: -0.030, val_auc: 0.769

Epoch 4/300, Step 920/920, Loss: -0.026
train_loss: -0.042, val_loss: -0.050, val_auc: 0.769

Epoch 5/300, Step 920/920, Loss: -0.047
train_loss: -0.064, val_loss: -0.074, val_auc: 0.769

Epoch 6/300, Step 920/920, Loss: -0.064
train_loss: -0.088, val_loss: -0.098, val_auc: 0.769

Epoch 7/300, Step 920/920, Loss: -0.073
train_loss: -0.111, val_loss: -0.120, val_auc: 0.769

Epoch 8/300, Step 920/920, Loss: -0.075
train_loss: -0.133, val_loss: -0.140, val_auc: 0.769

Epoch 9/300, Step 920/920, Loss: -0.070
train_loss: -0.152, val_loss: -0.157, val_auc: 0.769

Epoch 10/300, Step 920/920, Loss: -0.059
train_loss: -0.170, val_loss: -0.172, val_auc: 0.769

Ep

/usr/lib/python3.9/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


...

Fold 4 / 5, 18 cones
Using GPU
Epoch 1/300, Step 920/920, Loss: 0.0079
train_loss: -0.002, val_loss: -0.005, val_auc: 0.769

Epoch 2/300, Step 920/920, Loss: -0.004
train_loss: -0.009, val_loss: -0.013, val_auc: 0.769

Epoch 3/300, Step 920/920, Loss: -0.023
train_loss: -0.020, val_loss: -0.028, val_auc: 0.769

Epoch 4/300, Step 920/920, Loss: -0.048
train_loss: -0.036, val_loss: -0.046, val_auc: 0.769

Epoch 5/300, Step 920/920, Loss: -0.069
train_loss: -0.055, val_loss: -0.066, val_auc: 0.769

Epoch 6/300, Step 920/920, Loss: -0.084
train_loss: -0.074, val_loss: -0.084, val_auc: 0.769

Epoch 7/300, Step 920/920, Loss: -0.093
train_loss: -0.091, val_loss: -0.099, val_auc: 0.769

Epoch 8/300, Step 920/920, Loss: -0.097
train_loss: -0.105, val_loss: -0.113, val_auc: 0.769

Epoch 9/300, Step 920/920, Loss: -0.097
train_loss: -0.118, val_loss: -0.125, val_auc: 0.769

Epoch 10/300, Step 920/920, Loss: -0.093
train_loss: -0.129, val_loss: -0.135, val_auc: 0.769

Epoch 11/300, Step 920/920, 

/mnt/ntfs/home_backup/edwit4/Research/DoubleMuParked/cms_llp/llp_auc.py:360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  roc_dict[key] = np.array(roc_dict[key])


Fold 5 / 5, 18 cones
Using GPU
Epoch 1/300, Step 920/920, Loss: -0.004
train_loss: -0.001, val_loss: -0.003, val_auc: 0.771

Epoch 2/300, Step 920/920, Loss: -0.008
train_loss: -0.007, val_loss: -0.012, val_auc: 0.771

Epoch 3/300, Step 920/920, Loss: -0.012
train_loss: -0.018, val_loss: -0.027, val_auc: 0.771

Epoch 4/300, Step 920/920, Loss: -0.016
train_loss: -0.035, val_loss: -0.046, val_auc: 0.770

Epoch 5/300, Step 920/920, Loss: -0.022
train_loss: -0.053, val_loss: -0.066, val_auc: 0.770

Epoch 6/300, Step 920/920, Loss: -0.030
train_loss: -0.072, val_loss: -0.084, val_auc: 0.770

Epoch 7/300, Step 920/920, Loss: -0.041
train_loss: -0.088, val_loss: -0.100, val_auc: 0.770

Epoch 8/300, Step 920/920, Loss: -0.054
train_loss: -0.102, val_loss: -0.113, val_auc: 0.770

Epoch 9/300, Step 920/920, Loss: -0.069
train_loss: -0.114, val_loss: -0.124, val_auc: 0.770

Epoch 10/300, Step 920/920, Loss: -0.086
train_loss: -0.124, val_loss: -0.133, val_auc: 0.770

Epoch 11/300, Step 920/920, 

/mnt/ntfs/home_backup/edwit4/Research/DoubleMuParked/cms_llp/llp_auc.py:360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  roc_dict[key] = np.array(roc_dict[key])


LLP AUC: 0.7106085711602044
Average AUC: 0.7134520265207227 +- 0.027833247954685936


# Performance

### AUC Summary

In [9]:
for auc, sig, num_cones in zip(cone_set_aucs, cone_set_sigs, [1,18]):
    print(f'{num_cones} cones AUC: {auc} +- {sig}')

1 cones AUC: 0.848086044195483 +- 0.004699216467052448
18 cones AUC: 0.7134520265207227 +- 0.027833247954685936


### Valid Outputs - Largest Cone

On legends "signal" is majority signal (95% signal), "background" is majority background (20% signal)

In [26]:
valid_output_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('valid_output' in x) and ('1_cones' in x)]
ipyplot.plot_images(valid_output_images,img_width=300)

### Valid Outputs - All Cones

In [27]:
valid_output_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('valid_output' in x) and ('18_cones' in x)]
ipyplot.plot_images(valid_output_images,img_width=300)

### Test Outputs - Largest Cone

In [28]:
test_output_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_output' in x) and ('1_cones' in x)]
ipyplot.plot_images(test_output_images,img_width=300)

### Test Outputs - All Cones

In [29]:
test_output_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_output' in x) and ('18_cones' in x)]
ipyplot.plot_images(test_output_images,img_width=300)

### ROC Curve - Largest Cone

In [30]:
llp_roc_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('llp_roc' in x) and ('1_cones' in x)]
ipyplot.plot_images(llp_roc_images,img_width=300)

### ROC Curve - All Cones

In [31]:
llp_roc_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('llp_roc' in x) and ('18_cones' in x)]
ipyplot.plot_images(llp_roc_images,img_width=300)

### Loss History - Largest Cone

In [32]:
loss_history_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('loss_history' in x) and ('1_cones' in x)]
ipyplot.plot_images(loss_history_images,img_width=300)

### Loss History - All Cones

In [33]:
loss_history_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('loss_history' in x) and ('18_cones' in x)]
ipyplot.plot_images(loss_history_images,img_width=300)

### Signal Proportion Above Threshold - Largest Cone

In [34]:
test_frac_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_sig_frac' in x) and ('1_cones' in x)]
ipyplot.plot_images(test_frac_images,img_width=300)

### Signal Proportion Above Threshold - All Cones

In [35]:
test_frac_sigma_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_frac_sigma' in x) and ('1_cones' in x)]
ipyplot.plot_images(test_frac_sigma_images,img_width=300)

### Signal Proportion Std. Dev. - Largest Cone

In [36]:
test_frac_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_sig_frac' in x) and ('18_cones' in x)]
ipyplot.plot_images(test_frac_images,img_width=300)

### Signal Proportion Std. Dev. - All Cones

In [37]:
test_frac_sigma_images = [f'{image_folder}/{x}' for x in os.listdir(image_folder) if 
                  ('test_frac_sigma' in x) and ('18_cones' in x)]
ipyplot.plot_images(test_frac_sigma_images,img_width=300)

# Debugging

### Check data shapes

In [38]:
print(x.shape)
print(select_cone_set(x,18).shape)
print(select_cone_set(x,1).shape)
print(select_cone_set(x[train_index],18).shape)
print(select_cone_set(x[train_index],1).shape)
print(select_cone_set(x[valid_index],18).shape)
print(select_cone_set(x[valid_index],1).shape)

(147200, 18)
(147200, 18)
(147200, 1)
(117760, 18)
(117760, 1)
(29440, 18)
(29440, 1)


### Check preprocessing - 18 Cones

In [39]:
debug_train = {'x': select_cone_set(x[train_index],18), 
                      'f': f[train_index], 'y': y[train_index]}
debug_valid = {'x': select_cone_set(x[valid_index],18), 
                      'f': f[valid_index], 'y': y[valid_index]}
debug_test = {'x': select_cone_set(x_test,18),
                     'f': f_test, 'y': y_test, 'm': m_test}

print('Before preprocessing:')
print(debug_train['x'])
print('')
manual_x = (debug_train['x'] - np.mean(debug_train['x'], axis=0)) / np.std(debug_train['x'], axis=0)
print('After manual preprocessing:')
print(manual_x)
print('')
debug_train, debug_valid, debug_test = iso_standard_scaler(debug_train, debug_valid, debug_test)
print('After scikit preprocessing:')
print(debug_train['x'])
print('')
print('All elements match between methods?')
print(np.all(manual_x == debug_train['x']))

Before preprocessing:
[[1.03881079e+00 1.03881079e+00 1.03881079e+00 ... 1.62143552e-01
  1.52247402e-01 0.00000000e+00]
 [1.29244496e+00 1.26920478e+00 1.22570187e+00 ... 3.75824974e-01
  3.05820014e-01 4.00359768e-02]
 [4.02241225e+00 4.00705416e+00 3.93862231e+00 ... 2.54706716e+00
  2.35164959e+00 1.32495951e+00]
 ...
 [7.58893694e-02 6.97016814e-02 6.55898405e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.84335222e-01 1.64815126e-01 1.55270645e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.31885704e-02 5.62381267e-02 5.62381267e-02 ... 3.66512005e-03
  0.00000000e+00 0.00000000e+00]]

After manual preprocessing:
[[ 0.82884319  0.87958961  0.92987845 ...  0.13266261  0.29042151
  -0.16675095]
 [ 1.20330626  1.22524017  1.21471529 ...  0.71883432  0.83612329
   0.0893759 ]
 [ 5.23380401  5.33272223  5.34942138 ...  6.67499431  8.10573219
   8.30956782]
 ...
 [-0.59280484 -0.57432467 -0.55338753 ... -0.31213021 -0.25057129
  -0.16675095]
 [-0.43269642 -0.431

In [40]:
debug_train = {'x': select_cone_set(x[train_index],1), 
                      'f': f[train_index], 'y': y[train_index]}
debug_valid = {'x': select_cone_set(x[valid_index],1), 
                      'f': f[valid_index], 'y': y[valid_index]}
debug_test = {'x': select_cone_set(x_test,1),
                     'f': f_test, 'y': y_test, 'm': m_test}

print('Before preprocessing:')
print(debug_train['x'])
print('')
manual_x = (debug_train['x'] - np.mean(debug_train['x'], axis=0)) / np.std(debug_train['x'], axis=0)
print('After manual preprocessing:')
print(manual_x)
print('')
debug_train, debug_valid, debug_test = iso_standard_scaler(debug_train, debug_valid, debug_test)
print('After scikit preprocessing:')
print(debug_train['x'])
print('')
print('All elements match between methods?')
print(np.all(manual_x == debug_train['x']))

Before preprocessing:
[[1.03881079]
 [1.29244496]
 [4.02241225]
 ...
 [0.07588937]
 [0.18433522]
 [0.07318857]]

After manual preprocessing:
[[ 0.82884319]
 [ 1.20330626]
 [ 5.23380401]
 ...
 [-0.59280484]
 [-0.43269642]
 [-0.59679228]]

After scikit preprocessing:
[[ 0.82884319]
 [ 1.20330626]
 [ 5.23380401]
 ...
 [-0.59280484]
 [-0.43269642]
 [-0.59679228]]

All elements match between methods?
True
